# Car Classification: The image is processed and passed to the car_detection_model. If a car is detected with a confidence level of 0.5 or higher, the pipeline proceeds to the next steps.

# Brand Detection: The image is processed by brand_detection_model, and the predicted brand is displayed.

# Damage Detection: The image is processed by damage_detection_model. If damage is detected, the pipeline moves to location detection.

# Location Detection: The image is processed by location_detection_model to determine if the damage is at the front, side, or back of the car.

# Part Detection: The image is processed by the part_detection_model (PyTorch). Detected parts are displayed.

# Severity Check: The image is processed by severity_model to classify the severity of the damage.

# Price Prediction: The features collected from previous steps are passed into a price prediction model (price_model) to estimate the repair price.

In [6]:
import os
import numpy as np
import torch
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from torchvision.transforms import functional as F
from PIL import Image
import pickle
import warnings
warnings.filterwarnings("ignore")  # Ignore all warnings
import os
# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

# Load the TensorFlow models
car_detection_model = load_model('car_classification_model.h5')  # Step 1: Car classification
brand_detection_model = load_model('brand_detection_model.h5')  # Step 2: Brand detection
damage_detection_model = load_model('damage_detection_model.h5')  # Step 3: Damage detection
location_detection_model = load_model('location_detection_model.h5')  # Step 4: Location detection
# Step 5: Part Detection
part_detection_model = torch.load('Part_predection_model.pkl', map_location=torch.device('cpu'))
part_detection_model.eval()
# Step 6: Severity check
severity_model = load_model('severity_model.h5')
# Step 7: Price Predection 
with open('price_P_model.pkl', 'rb') as file:
    price_model = pickle.load(file)

# Hardcoded label map for part detection
label_map = {
    0: "background", 1: "Damage-Windshield", 2: "Damage-boots", 3: "Damage-door",
    4: "Damage-front-bumper", 5: "Damage-headlight", 6: "Damage-hood",
    7: "Damage-left-fender", 8: "Damage-rear-bumper", 9: "Damage-rear-light",
    10: "Damage-right-fender", 11: "Damage-roof", 12: "Damage-side-mirror",
    13: "Damage-window-glass", 14: "flat-tire"
}

# Extract categories (class labels) for brand detection
brand_categories = ["Ford_Mustang", "Mazda_3", "Toyota_Camry"]

# Helper Functions
def preprocess_image_tensorflow(img_path, img_width, img_height):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img) / 255.0
    return np.expand_dims(img_array, axis=0)

def preprocess_image_pytorch(img_path):
    img = Image.open(img_path).convert("RGB")
    img_tensor = F.to_tensor(img)
    return img_tensor, img

def predict_brand(img_path, model, categories):
    img = preprocess_image_tensorflow(img_path, IMG_WIDTH, IMG_HEIGHT)
    predictions = model.predict(img)
    predicted_class_idx = np.argmax(predictions[0])
    return categories[predicted_class_idx]

# Main Pipeline
def complete_detection_pipeline(img_path, confidence_threshold=0.5):
    print("Processing image:", img_path)
    features = {}  # Collect features for price prediction
    
    # Step 1: Car classification
    car_input_shape = car_detection_model.input_shape[1:3]
    car_image = preprocess_image_tensorflow(img_path, *car_input_shape)
    car_prediction = car_detection_model.predict(car_image)[0][0]
    features['car_detected'] = car_prediction
    if car_prediction >= 0.5:
        print(f"Step 1: Car classification - Detected with confidence {car_prediction:.2f}")
        
        # Step 2: Brand detection
        brand_prediction = predict_brand(img_path, brand_detection_model, brand_categories)
        features['brand'] = brand_prediction
        print(f"Step 2: Brand detection - Detected brand: {brand_prediction}")
        
        # Step 3: Damage detection
        damage_input_shape = damage_detection_model.input_shape[1:3]
        damage_image = preprocess_image_tensorflow(img_path, *damage_input_shape)
        damage_prediction = damage_detection_model.predict(damage_image)[0][0]
        features['damage_detected'] = damage_prediction
        if damage_prediction >= 0.5:
            print(f"Step 3: Damage detection - Damage detected with confidence {damage_prediction:.2f}")
            
            # Step 4: Location detection
            location_input_shape = location_detection_model.input_shape[1:3]
            location_image = preprocess_image_tensorflow(img_path, *location_input_shape)
            location_prediction = location_detection_model.predict(location_image)
            location_class = np.argmax(location_prediction)
            location_confidence = location_prediction[0][location_class]
            location_labels = ['front', 'side', 'back']
            features['location'] = location_labels[location_class]
            print(f"Step 4: Location detection - Location: {location_labels[location_class]} with confidence {location_confidence:.2f}")
            
            # Step 5: Part detection
            print("Step 5: Part detection")
            img_tensor, _ = preprocess_image_pytorch(img_path)
            img_tensor = img_tensor.unsqueeze(0)
            with torch.no_grad():
                outputs = part_detection_model(img_tensor)[0]
                outputs = {k: v for k, v in outputs.items() if len(v) > 0}
                if outputs:
                    parts_detected = []
                    for i, label in enumerate(outputs['labels']):
                        score = outputs['scores'][i].item()
                        if score >= confidence_threshold:
                            part_name = label_map[label.item()]
                            parts_detected.append(part_name)
                    features['parts_detected'] = parts_detected
                    print("Detected parts:", parts_detected)
                else:
                    features['parts_detected'] = []
                    print("No parts detected above the confidence threshold.")
            
            # Step 6: Severity check
            severity_input_shape = severity_model.input_shape[1:3]
            severity_image = preprocess_image_tensorflow(img_path, *severity_input_shape)
            severity_prediction = severity_model.predict(severity_image)
            severity_class = np.argmax(severity_prediction)
            severity_confidence = severity_prediction[0][severity_class]
            severity_labels = ['minor', 'moderate', 'severe']
            features['severity'] = severity_labels[severity_class]
            print(f"Step 6: Severity check - Severity: {severity_labels[severity_class]} with confidence {severity_confidence:.2f}")
            
            # Step 7: Price prediction
            input_vector = np.zeros(23)  # Replace 23 with the exact number of features used in training
            
            # Fill in features (ensure alignment with training data)
            input_vector[0] = features.get('car_detected', 0)  # Car classification confidence
            input_vector[1] = 1 if features.get('brand', '') == "Mazda_3" else 0  # Brand encoding example
            input_vector[2] = 1 if features.get('brand', '') == "Ford_Mustang" else 0
            input_vector[3] = 1 if features.get('brand', '') == "Toyota_Camry" else 0
            input_vector[4] = features.get('damage_detected', 0)  # Damage detection confidence
            input_vector[5] = 1 if features.get('location', '') == "front" else 0  # Location encoding example
            input_vector[6] = 1 if features.get('location', '') == "side" else 0
            input_vector[7] = 1 if features.get('location', '') == "back" else 0
            input_vector[8] = severity_confidence  # Severity confidence
            
            # Parts detected (binary indicators)
            # Parts detected (binary indicators)
            for part_index, part_name in label_map.items():
                # Ensure part_name is a string and properly checked
                if isinstance(part_name, str) and part_name.startswith("Damage-"):
                    if part_name in features.get('parts_detected', []):
                        input_vector[9 + part_index] = 1  # Adjust indices to align with training data

            # Reshape for prediction
            input_vector = input_vector.reshape(1, -1)
            predicted_price = np.expm1(price_model.predict(input_vector)[0])
            print(f"Step 7: Price prediction - Estimated price: ${predicted_price:.2f}")
        else:
            print(f"Step 3: Damage detection - No damage detected with confidence {1 - damage_prediction:.2f}")
    else:
        print(f"Step 1: Car classification - No car detected with confidence {1 - car_prediction:.2f}")

# Parameters
IMG_HEIGHT, IMG_WIDTH = 128, 128

# Example usage
test_image_path = r"E:\car_crash\test\test7.jpg"  # Replace with your test image path
complete_detection_pipeline(test_image_path, confidence_threshold=0.5)

Processing image: E:\car_crash\test\test7.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Step 1: Car classification - Detected with confidence 0.93
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Step 2: Brand detection - Detected brand: Toyota_Camry
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Step 3: Damage detection - Damage detected with confidence 0.78
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
Step 4: Location detection - Location: front with confidence 0.56
Step 5: Part detection
Detected parts: ['Damage-front-bumper', 'Damage-right-fender', 'Damage-boots', 'Damage-hood']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
Step 6: Severity check - Severity: severe with confidence 0.81
Step 7: Price prediction - Estimated price: $146.90


In [2]:
pip freeze

absl-py==2.1.0
aiobotocore @ file:///C:/b/abs_3cwz1w13nn/croot/aiobotocore_1701291550158/work
aiohttp @ file:///C:/b/abs_27h_1rpxgd/croot/aiohttp_1707342354614/work
aioitertools @ file:///tmp/build/80754af9/aioitertools_1607109665762/work
aiosignal @ file:///tmp/build/80754af9/aiosignal_1637843061372/work
alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
altair @ file:///C:/b/abs_27reu1igbg/croot/altair_1687526066495/work
anaconda-anon-usage @ file:///C:/b/abs_95v3x0wy8p/croot/anaconda-anon-usage_1697038984188/work
anaconda-catalogs @ file:///C:/b/abs_8btyy0o8s8/croot/anaconda-catalogs_1685727315626/work
anaconda-client @ file:///C:/b/abs_34txutm0ue/croot/anaconda-client_1708640705294/work
anaconda-cloud-auth @ file:///C:/b/abs_410afndtyf/croot/anaconda-cloud-auth_1697462767853/work
anaconda-navigator @ file:///C:/b/abs_cfvv8k_j21/croot/anaconda-navigator_1704813334508/work
anaconda-project @ file:///C:/ci_311/anaconda-project_1676458365912/work
anyascii==0.3.2
anyio 